In [4]:
# MNIST Dataset
# datasets contain 70000 small images of digits handwritting

from six.moves import urllib
from sklearn.datasets import fetch_mldata

from scipy.io import loadmat, savemat
# mnist_alternative_url = "https://github.com/amplab/datascience-sp14/raw/master/lab7/mldata/mnist-original.mat"
# mnist_path = "./mnist-original.mat"
# response = urllib.request.urlopen(mnist_alternative_url)
# with open(mnist_path, "wb") as f:
#     content = response.read()
#     f.write(content)
mnist_raw = loadmat('mnist.mat')
# savemat('mnist.mat', loadmat(mnist_path))
mnist = {
        "data": mnist_raw["data"].T,
        "target": mnist_raw["label"][0],
        "COL_NAMES": ["label", "data"],
        "DESCR": "mldata.org dataset: mnist-original",
    }


In [5]:
X, y = mnist["data"], mnist["target"]
X.shape
y.shape

(70000,)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

some_digit = X[36000]
some_digit_image = some_digit.reshape(28,28)

plt.imshow(some_digit_image, cmap = matplotlib.cm.binary,
          interpolation="nearest")
plt.axis("off")
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = X[:60000], X[10000:], y[:60000], y[10000:]

import numpy as np

# shuffling the data
shuffle_index = np.random.permutation(60000)
X_train, y_train = X_train[shuffle_index], y_train[shuffle_index]


In [ ]:
# creating 5-detector
y_train_5 = (y_train == 5.0)
y_test_5 = (y_test == 5.0)

from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train_5)

sgd_clf.predict([some_digit])


In [ ]:
# Using stratified K folds to estimate the accuracy
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone

skfolds = StratifiedKFold(n_splits=3, random_state=42)

for train_index, test_index in skfolds.split(X_train, y_train_5):
    clone_clf = clone(sgd_clf)
    X_train_folds = X_train[train_index]
    y_train_folds = y_train_5[train_index]
    X_test_folds = X_test[test_index]
    y_test_folds = y_test_5[test_index]
    
    clone_clf.fit(X_train_folds, y_train_folds)
    y_pred = clone_clf.predict(X_test_folds)
    n_correct = sum(y_pred == y_test_folds)
    print(n_correct/len(y_pred))
    

In [ ]:
# Cross validation
from sklearn.model_selection import cross_val_score
cross_val_score(sgd_clf, X_train, y_train_5, cv=3, scoring="accuracy")


In [ ]:
from sklearn.base import BaseEstimator

class Never5Classifier(BaseEstimator):
    def fit(self, X, y=None):
        pass
    def predict(self, X):
        return np.zeros((len(X), 1), dtype=bool)

    
    
never_5_clf = Never5Classifier()
cross_val_score(never_5_clf, X_train, y_test_5, cv=3, scoring="accuracy")

In [ ]:
## CONFUSION MATRIX

from sklearn.metrics import confusion_matrix
confusion_matrix(y_train_5, y_train_pred)